# Huffman树构建

Huffman树的节点：

In [0]:
import numpy as np

class HuffmanNode:
    def __init__(self, is_leaf=False, value=None, fre=0,
                 left=None, right=None):
        """
        如果是中间节点的话，value储存的是中间向量
        如果是叶子节点的话，value储存的是word的index
        """
        self.is_leaf = is_leaf
        self.value = value
        self.fre = fre
        self.huffmancode = ""
        self.left = left
        self.right = right

Huffman树类：

In [0]:
class Huffman:
    def __init__(self, freq_dict, embed_size):
        self.root = None
        self.freq_dict = freq_dict
        self.embed_size = embed_size
        self.huffman_code = {}
        node_list = [HuffmanNode(is_leaf=True, value=word, fre=fre) for word, fre in freq_dict.items()]
        self.build_tree(node_list)
        self.generate_huffman_code()
    
    def build_tree(self, node_list):
        while len(node_list) > 1:
            # 找出frequencies最小的两个nodes
            index1 = 0
            index2 = 1
            if node_list[index1].fre > node_list[index2].fre:
                index1, index2 = index2, index1
            for index in range(2, len(node_list)):
                if node_list[index].fre < node_list[index2].fre:
                    index2 = index
                    if node_list[index1].fre > node_list[index2].fre:
                        index1, index2 = index2, index1
            
            parentnode = HuffmanNode(is_leaf=False,
                                     value=np.random.normal(0, 0.1, size=(1, self.embed_size)),
                                     fre=node_list[index1].fre+node_list[index2].fre,
                                     left=node_list[index2], right=node_list[index1])
            
            if index1 > index2:
                node_list.pop(index1)
                node_list.pop(index2)
            else:
                node_list.pop(index2)
                node_list.pop(index1)
            
            node_list.append(parentnode)
        self.root=node_list[0]
    
    def generate_huffman_code(self):
        """
        用DFS构建HuffmanCode
        """
        stack = [self.root]
        while stack:
            node = stack.pop()
            while node.left:
                code = node.huffmancode
                node.left.huffmancode = code + '1'
                if node.right:
                    node.right.huffmancode = code + '0'
                    stack.append(node.right)
                node = node.left
            if not node.right:
                self.huffman_code[node.value] = node.huffmancode
            else:
                node.right.huffmancode = code + '0'
                stack.append(node.right)

In [0]:
text = ['我', '喜欢', '玩', '手机', '游戏', '绝地求生']
fre = [15, 9, 4, 8, 2, 1]
word_dict = {word: f for word, f in zip(text, fre)}

In [192]:
h = Huffman(word_dict, 10)
h.huffman_code

{'喜欢': '10', '我': '11', '手机': '01', '游戏': '0001', '玩': '001', '绝地求生': '0000'}

# Word2Vec函数构建

In [0]:
class W2V:
    def __init__(self, vocab_size, embed_size, learning_rate, model_type,
                 HuffmanTree):
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.learning_rate = learning_rate
        self.model_type = model_type
        self.embedding_matrix = np.random.normal(0, 0.1, size=(vocab_size, embed_size))
        self.HuffmanTree = HuffmanTree
    
    def train(self, target, context):
        if self.model_type == 'CBOW':
            self._CBOW(target, context)
        elif self.model_type == 'SkipGram':
            self._SkipGram(target, context)

    def _sigmoid(self, a):
        return 1.0 / (1.0 + np.exp(-a))

    def _CBOW(self, target, context):
        target_huffman = self.HuffmanTree.huffman_code[target]
        Xw = np.zeros(shape=(1, self.embed_size))

        for u in context:
            Xw += self.embedding_matrix[u]

        e = self._search_along_huffman(Xw, target_huffman)

        for u in context:
            self.embedding_matrix[u] += e[0]
    
    def _SkipGram(self, target, context):
        Vw = self.embedding_matrix[target].reshape(1, -1)
        e = np.zeros(shape=[1, self.embed_size])
        for u in context:
            e += self._search_along_huffman(Vw, self.HuffmanTree.huffman_code[u])
        self.embedding_matrix[target] += e[0]


    def _search_along_huffman(self, Xw, code):
        e = np.zeros(shape=[1, self.embed_size])
        node = self.HuffmanTree.root
        for level in code:
            q = self._sigmoid(Xw @ node.value.T)
            g = self.learning_rate * (1 - int(level) - q)
            e = e + g * node.value
            node.value += g * Xw
            if level == '1':
                node = node.left
            else:
                node = node.right
        return e

# 训练数据读取

In [87]:
import os
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [88]:
path = '/content/drive/My Drive'
os.chdir(path)
!ls

 2238322.gdoc	   'Modern Family s01e01 Episode Script | SS.gdoc'
 2238322.pdf	   'Modern Family s01e01 Episode Script | SS.pdf'
'Colab Notebooks'   ptb.train.txt
 fractal.mp4	    WechatIMG7.jpeg


In [0]:
with open('ptb.train.txt', encoding='utf-8') as f:
    lines = f.readlines()
raw_dataset = [line.split() for line in lines]

#词频统计并去低频词

In [0]:
from collections import Counter

counter = Counter([word for line in raw_dataset for word in line])
counter = {word: fre for word, fre in counter.items() if fre > 5}

#构建词语与index的映射

In [0]:
idx_to_word = [word for word, _ in counter.items()]
word_to_idx = {word: idx for idx, word in enumerate(idx_to_word)}

#更新数据集

In [0]:
datasets = [[word_to_idx[word] for word in line if word in word_to_idx] for line in raw_dataset]
counter = {word_to_idx[word]: fre for word, fre in counter.items()}

#Context与Target生成函数

In [0]:
import random

def get_center_context(text, max_window_size):
    center = []
    contexts = []
    for line in text:
        center += line
        for center_i in range(len(line)):
            window_size = random.randint(1, max_window_size)
            indices = list(range(max(0, center_i - window_size), min(len(line), center_i + window_size + 1)))
            indices.remove(center_i)
            contexts.append([line[idx] for idx in indices])
    return center, contexts

#模型训练

In [0]:
center, contexts = get_center_context(datasets, 5)
htree = Huffman(counter, 100)
# CBOW
model = W2V(len(counter), 100, 0.01, "CBOW", htree)
# SkipGram
model2 = W2V(len(counter), 100, 0.01, "SkipGram", htree)

In [144]:
import time
index = 1
start = time.time()
for target, context in zip(center, contexts):
    if index % 100000 == 0:
        print(f"{index}/{len(center)} target words training finish. Time: {time.time()-start:.3f}s")
        start = time.time()
    model.train(target, context)
    index += 1

100000/885720 target words training finish. Time: 16.496s
200000/885720 target words training finish. Time: 16.430s
300000/885720 target words training finish. Time: 16.421s
400000/885720 target words training finish. Time: 16.382s
500000/885720 target words training finish. Time: 16.291s
600000/885720 target words training finish. Time: 17.316s
700000/885720 target words training finish. Time: 16.218s
800000/885720 target words training finish. Time: 16.281s


In [148]:
index = 1
start = time.time()
for target, context in zip(center, contexts):
    if index % 100000 == 0:
        print(f"{index}/{len(center)} target words training finish. Time: {time.time()-start:.3f}s")
        start = time.time()
    model2.train(target, context)
    index += 1

100000/885720 target words training finish. Time: 76.901s
200000/885720 target words training finish. Time: 77.514s
300000/885720 target words training finish. Time: 77.470s
400000/885720 target words training finish. Time: 76.767s
500000/885720 target words training finish. Time: 76.799s
600000/885720 target words training finish. Time: 76.750s
700000/885720 target words training finish. Time: 77.548s
800000/885720 target words training finish. Time: 77.546s


#词相似性计算

In [0]:
def get_similarity(word, k, embed):
    idx = word_to_idx[word]
    W = embed
    x = embed[idx].reshape(1, -1)
    cos = (W * x).sum(axis=-1) / np.sqrt((W * W).sum(axis=-1) * (x * x).sum(axis=-1))
    cos_indice = [(index, value) for index, value in enumerate(cos)]
    cos_indice = sorted(cos_indice, key=lambda x: x[1], reverse=True)
    for i in range(1, k+1):
        print(f"Cosine similarity = {cos_indice[i][1]:.3f}: {idx_to_word[cos_indice[i][0]]}")

In [193]:
print("CBOW Hierarchical Softmax:")
get_similarity('she', 4, model.embedding_matrix)

CBOW Hierarchical Softmax:
Cosine similarity = 0.715: he
Cosine similarity = 0.611: it
Cosine similarity = 0.528: ms.
Cosine similarity = 0.526: mrs.


In [194]:
print("SkipGram Hierarchical Softmax:")
get_similarity('she', 4, model2.embedding_matrix)

SkipGram Hierarchical Softmax:
Cosine similarity = 0.652: ms.
Cosine similarity = 0.645: her
Cosine similarity = 0.638: mrs.
Cosine similarity = 0.624: he
